In [7]:
import numpy as np
import pandas as pd 
import os
from pandas.tseries.holiday import USFederalHolidayCalendar
from glob import glob

import utils
# from utils import load_data, get_train_val_split, get_stratified_splitter
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedGroupKFold, train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer

from lightgbm import LGBMRegressor
from scipy.stats import kstest, kruskal, mannwhitneyu
from itertools import combinations
from collections import defaultdict
from tqdm import tqdm
from sklearn.ensemble import StackingRegressor
import re
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_log_error, mean_squared_error

import optuna
import pickle

In [10]:
import importlib
importlib.reload(utils)

<module 'utils' from '/Users/sharaddargan/Documents/Course Content/Spring 2023/Probability and Statistics 2/Project/EnergyPrediction-ASHRAE/code/utils.py'>

In [9]:
data_dict = utils.load_data('ashrae-energy-prediction')

Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.88%
Memory usage of dataframe is 9.60 MB
Memory usage after optimization is: 3.07 MB
Decreased by 68.05%


/Users/sharaddargan/Documents/Course Content/Spring 2023/Probability and Statistics 2/Project/EnergyPrediction-ASHRAE/code/utils.py:143: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  weather_train['timestamp'] = pd.to_datetime(weather_train['timestamp'], infer_datetime_format = True, utc = True).astype('datetime64[ns]')


Memory usage of dataframe is 19.04 MB
Memory usage after optimization is: 5.13 MB
Decreased by 73.04%


/Users/sharaddargan/Documents/Course Content/Spring 2023/Probability and Statistics 2/Project/EnergyPrediction-ASHRAE/code/utils.py:151: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  train['timestamp'] = pd.to_datetime(train['timestamp'], infer_datetime_format = True, utc = True).astype('datetime64[ns]')


Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 289.19 MB
Decreased by 53.12%
Memory usage of dataframe is 1272.51 MB
Memory usage after optimization is: 358.53 MB
Decreased by 71.82%


/Users/sharaddargan/Documents/Course Content/Spring 2023/Probability and Statistics 2/Project/EnergyPrediction-ASHRAE/code/utils.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[(train['site_id'] == 0) & (train['meter'] == 0)]['meter_reading'] = 0.2931 * train[(train['site_id'] == 0) & (train['meter'] == 0)]['meter_reading']


In [11]:
# Add weather features 
weather_features = ['cloud_coverage', 'dew_temperature', 'air_temperature', 
                    'sea_level_pressure', 'wind_direction', 'wind_speed', 'precip_depth_1_hr',]

hourly_by_site = data_dict["X_train"].groupby(['hour', 'month', 'site_id'])[weather_features].mean().reset_index()

data_dict["X_train"] = data_dict["X_train"].merge(
    hourly_by_site, 
    on=['hour', 'month', 'site_id'], 
    how='left', 
    suffixes=(None, '_hourly_by_site')
)

del hourly_by_site

for feature in weather_features:
    # Fill in NA values from weather with hourly by site columns 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature + "_hourly_by_site"],
        inplace=True
    )
    
    # Fill in the rest with the median 
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(),
        inplace=True
    )
    
    data_dict["X_train"][feature + "_diff_hourly_from_mean"] = data_dict["X_train"][feature] - \
        data_dict["X_train"][feature + "_hourly_by_site"]
    
data_dict["X_train"] = data_dict["X_train"].drop(columns = [feat + "_hourly_by_site" for feat in weather_features])

In [12]:
# Fill in NA with median values for floor count and year_built
for feature in ['year_built', 'floor_count']:
    data_dict["X_train"][feature].fillna(
        data_dict["X_train"][feature].median(), 
        inplace=True
    )

In [13]:
data_dict.keys()

dict_keys(['weather_test', 'X_train', 'X_test', 'y_train'])

## Examine Differences (Non-Parametric)
Using Bonferonni's Correction

### Milestone 2. 
- Show difference in sites across meter readings 
- get average meter reading per day per site 
- conduct a (non-parametric ANOVA) KS OR pairs (mann-whitney) to show that they are diff 
- train a model per site id (with rudimentary hyperparameter tuning) 
- John sites 0-7, Sharad sites 8-15 

### Milestone 3. Determine, per site, which primary uses are similar (if they have only a few buildings), which are diff
- for a given primary use, if diff, identify "clusters" of buildings that are similar 

In [14]:
features = ['year_built', 'floor_count', 'air_temperature',
       'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr',
       'sea_level_pressure', 'wind_direction', 'wind_speed',
       'air_temperature_mean_lag7', 'air_temperature_max_lag7',
       'air_temperature_min_lag7', 'air_temperature_std_lag7',
       'cloud_coverage_mean_lag7', 'cloud_coverage_max_lag7',
       'cloud_coverage_min_lag7', 'cloud_coverage_std_lag7',
       'dew_temperature_mean_lag7', 'dew_temperature_max_lag7',
       'dew_temperature_min_lag7', 'dew_temperature_std_lag7',
       'precip_depth_1_hr_mean_lag7', 'precip_depth_1_hr_max_lag7',
       'precip_depth_1_hr_min_lag7', 'precip_depth_1_hr_std_lag7',
       'sea_level_pressure_mean_lag7', 'sea_level_pressure_max_lag7',
       'sea_level_pressure_min_lag7', 'sea_level_pressure_std_lag7',
       'wind_direction_mean_lag7', 'wind_direction_max_lag7',
       'wind_direction_min_lag7', 'wind_direction_std_lag7',
       'wind_speed_mean_lag7', 'wind_speed_max_lag7', 'wind_speed_min_lag7',
       'wind_speed_std_lag7', 'log_square_feet', 'weekday', 'hour', 'day',
       'weekend', 'month', 'primary_use_enc']

In [15]:
X, y = data_dict['X_train'].loc[:, features+['meter', 'site_id', 'building_id']], data_dict['y_train']
splitter_gen = utils.get_stratified_splitter(X, y)
train_index, test_index = next(splitter_gen)
X_train, y_train, X_test, y_test = X.loc[train_index, :], y[train_index], X.loc[test_index, :], y[test_index]

/opt/homebrew/Caskroom/miniforge/base/envs/mlp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:885: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(


In [23]:
def run_optuna_search_cv(
    site: int,
    meter: int,
    X_train: pd.DataFrame,
    y_train: pd.DataFrame,
    features: list,
    n_trials: int = 50,
    
):
    """
    Runs Optuna Search for LGBMRegressor
    """
    SITE_FILTER = X_train["site_id"] == site
    METER_FILTER = X_train["meter"] == meter
    
    X, y = X_train.loc[SITE_FILTER&METER_FILTER, features], y_train[SITE_FILTER&METER_FILTER]
    if X.shape[0] == 0:
        return None

    splitter_gen = utils.get_stratified_splitter(X_train[SITE_FILTER&METER_FILTER], y_train[SITE_FILTER&METER_FILTER])

    

    regressor = LGBMRegressor()

    param_distributions = {
        "max_depth": optuna.distributions.IntDistribution(-1, len(X_train.columns)),
        "num_leaves": optuna.distributions.IntDistribution(5, 50),
        "learning_rate": optuna.distributions.FloatDistribution(1e-7, 1, log=True),
        "n_estimators": optuna.distributions.IntDistribution(1, 500),
        "reg_alpha": optuna.distributions.FloatDistribution(1e-7, 1e7, log=True),
        "reg_lambda": optuna.distributions.FloatDistribution(1e-7, 1e7, log=True),            
    }

    """
    Parameters not searched over: 
    subsample_for_bin: int = 200000,
    min_split_gain: float = 0.0,
    min_child_weight: float = 0.001,
    min_child_samples: int = 20,
    subsample: float = 1.0,
    subsample_freq: int = 0,
    colsample_bytree: float = 1.0,
    random_state: Union[int, numpy.random.mtrand.RandomState, NoneType] = None,
    n_jobs: int = -1,
    """
    def rmse(estimator, X_test, y_test):
        y_pred = estimator.predict(X_test)
        return -1 * mean_squared_error(y_test, y_pred, squared=False)
    
    optuna_search = optuna.integration.OptunaSearchCV(
        regressor, 
        param_distributions,
        n_trials=n_trials,
        cv = splitter_gen,
        random_state=0, # IMPORTANT,
        refit=True,
        n_jobs=4,
        scoring = rmse,
        verbose=0
    )

    optuna_search.fit(X, y)
    y_pred = optuna_search.predict(X)
    
    return optuna_search

In [24]:
# This fits sites 0-7
N_TRIALS = 50
models = defaultdict(dict)

for site in tqdm(X_train["site_id"].unique()):
    for meter in X_train["meter"].unique():
        
        optuna_search = run_optuna_search_cv(site, meter, X_train, y_train, features, N_TRIALS)
        
        model_identifier = "_".join([str(site), str(meter)])
        models[model_identifier] = optuna_search

  0%|                                                                             | 0/16 [00:00<?, ?it/s]/var/folders/6v/12r4qnjx54zgmfy66mv7j0rr0000gn/T/ipykernel_49269/2838603844.py:51: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
/opt/homebrew/Caskroom/miniforge/base/envs/mlp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:885: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
[I 2023-04-25 17:09:36,463] A new study created in memory with name: no-name-3f320d4b-1a62-4759-ba11-5b88010a3cfc
[I 2023-04-25 17:09:44,746] Trial 1 finished with value: -1.1910817565532403 and parameters: {'max_depth': 28, 'num_leaves': 17, 'learning_rate': 0.3451915537647719, 'n_estimators': 409, 'reg_alpha': 176562.08317097957, 'reg_lambda': 4.520273143539669}. Best is trial 1 with value: -1.191081756553240

[I 2023-04-25 17:11:18,309] Trial 25 finished with value: -1.152933282547638 and parameters: {'max_depth': 28, 'num_leaves': 11, 'learning_rate': 0.0032537985961924986, 'n_estimators': 77, 'reg_alpha': 0.0150169065006113, 'reg_lambda': 1.031204298609369e-07}. Best is trial 24 with value: -1.1516425730397934.
[I 2023-04-25 17:11:24,590] Trial 10 finished with value: -1.2748405304620383 and parameters: {'max_depth': 41, 'num_leaves': 34, 'learning_rate': 0.10804922264908694, 'n_estimators': 435, 'reg_alpha': 0.00022321383794605436, 'reg_lambda': 25649.509360594984}. Best is trial 24 with value: -1.1516425730397934.
[I 2023-04-25 17:11:24,909] Trial 28 finished with value: -1.1720080148264012 and parameters: {'max_depth': 24, 'num_leaves': 20, 'learning_rate': 0.007684418396927873, 'n_estimators': 61, 'reg_alpha': 2.2529422916562397e-07, 'reg_lambda': 2.848516704967427e-06}. Best is trial 24 with value: -1.1516425730397934.
[I 2023-04-25 17:11:25,656] Trial 26 finished with value: -1.1545

/var/folders/6v/12r4qnjx54zgmfy66mv7j0rr0000gn/T/ipykernel_49269/2838603844.py:51: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
/opt/homebrew/Caskroom/miniforge/base/envs/mlp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:885: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
[I 2023-04-25 17:12:58,275] A new study created in memory with name: no-name-72ad3894-42e9-455b-9984-4166f02d896c
[I 2023-04-25 17:12:58,818] Trial 1 finished with value: -2.212462479464853 and parameters: {'max_depth': 17, 'num_leaves': 44, 'learning_rate': 3.655561496990226e-05, 'n_estimators': 23, 'reg_alpha': 7477681.917773894, 'reg_lambda': 6108609.5475227535}. Best is trial 1 with value: -2.212462479464853.
[I 2023-04-25 17:13:02,086] Trial 2 finished with value: -2.2123493952620867 and parameters: {'max_dep

[I 2023-04-25 17:14:13,339] Trial 25 finished with value: -2.195713362885132 and parameters: {'max_depth': 0, 'num_leaves': 11, 'learning_rate': 6.19277030972353e-05, 'n_estimators': 494, 'reg_alpha': 3.0259168734632356, 'reg_lambda': 0.0025557275339493273}. Best is trial 9 with value: -2.135355489783563.
[I 2023-04-25 17:14:15,792] Trial 23 finished with value: -2.5350676612229437 and parameters: {'max_depth': -1, 'num_leaves': 19, 'learning_rate': 0.021037838022885075, 'n_estimators': 500, 'reg_alpha': 6.483615960311362, 'reg_lambda': 1.0034418621937218e-07}. Best is trial 9 with value: -2.135355489783563.
[I 2023-04-25 17:14:16,474] Trial 26 finished with value: -2.190577333796612 and parameters: {'max_depth': -1, 'num_leaves': 11, 'learning_rate': 8.161691492211642e-05, 'n_estimators': 500, 'reg_alpha': 1.1955389253156536, 'reg_lambda': 1.4312129303269396}. Best is trial 9 with value: -2.135355489783563.
[I 2023-04-25 17:14:27,650] Trial 27 finished with value: -2.1313487453039666 

  6%|████▏                                                             | 1/16 [06:53<1:43:20, 413.37s/it]/var/folders/6v/12r4qnjx54zgmfy66mv7j0rr0000gn/T/ipykernel_49269/2838603844.py:51: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  optuna_search = optuna.integration.OptunaSearchCV(
/opt/homebrew/Caskroom/miniforge/base/envs/mlp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:885: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  warnings.warn(
[I 2023-04-25 17:16:31,092] A new study created in memory with name: no-name-6fd87fed-07c1-41e6-83be-45fd3a1a9fa2
[I 2023-04-25 17:16:47,452] Trial 3 finished with value: -1.3443181323458302 and parameters: {'max_depth': 47, 'num_leaves': 21, 'learning_rate': 4.1772734244382633e-07, 'n_estimators': 317, 'reg_alpha': 1535099.2379272394, 'reg_lambda': 0.007550574599091759}. Best is trial 3 with value: -1.34431813

...

[I 2023-04-25 20:39:50,112] Trial 31 finished with value: -1.7313766744317505 and parameters: {'max_depth': 4, 'num_leaves': 8, 'learning_rate': 0.019110218340461605, 'n_estimators': 425, 'reg_alpha': 90.79204543490422, 'reg_lambda': 3.883335810698204e-06}. Best is trial 34 with value: -1.6660006459313754.
[I 2023-04-25 20:39:51,095] Trial 33 finished with value: -1.7820885286099144 and parameters: {'max_depth': 34, 'num_leaves': 5, 'learning_rate': 0.21992712371691528, 'n_estimators': 412, 'reg_alpha': 1.0906002861250804e-05, 'reg_lambda': 0.00037838077690848816}. Best is trial 34 with value: -1.6660006459313754.
[I 2023-04-25 20:39:55,680] Trial 39 finished with value: -1.789557129035493 and parameters: {'max_depth': 25, 'num_leaves': 15, 'learning_rate': 0.21944644022305237, 'n_estimators': 105, 'reg_alpha': 3.8077895922019057e-06, 'reg_lambda': 0.001907098893971074}. Best is trial 34 with value: -1.6660006459313754.
[I 2023-04-25 20:39:57,043] Trial 41 finished with value: -1.77336

[I 2023-04-25 20:41:33,799] Trial 13 finished with value: -1.1790196141047364 and parameters: {'max_depth': 19, 'num_leaves': 5, 'learning_rate': 0.9704115560696056, 'n_estimators': 184, 'reg_alpha': 458.5175316895456, 'reg_lambda': 0.0003757295381893269}. Best is trial 4 with value: -0.9718200808825319.
[I 2023-04-25 20:41:54,536] Trial 14 finished with value: -1.2696156224522746 and parameters: {'max_depth': 29, 'num_leaves': 34, 'learning_rate': 0.3709593024434252, 'n_estimators': 495, 'reg_alpha': 75.94558232111268, 'reg_lambda': 0.029329875621029938}. Best is trial 4 with value: -0.9718200808825319.
[I 2023-04-25 20:42:06,145] Trial 10 finished with value: -1.1913470486684883 and parameters: {'max_depth': 33, 'num_leaves': 49, 'learning_rate': 0.0026357756506362367, 'n_estimators': 332, 'reg_alpha': 6.583961678373859e-07, 'reg_lambda': 0.0051184387334183784}. Best is trial 4 with value: -0.9718200808825319.
[I 2023-04-25 20:42:09,127] Trial 11 finished with value: -1.0934752617790

[I 2023-04-25 20:43:22,910] Trial 39 finished with value: -1.0466249802286065 and parameters: {'max_depth': 35, 'num_leaves': 11, 'learning_rate': 0.6415227526006875, 'n_estimators': 156, 'reg_alpha': 483.1146805463328, 'reg_lambda': 0.10524563853521764}. Best is trial 4 with value: -0.9718200808825319.
[I 2023-04-25 20:43:25,712] Trial 38 finished with value: -1.2268979031673197 and parameters: {'max_depth': 45, 'num_leaves': 45, 'learning_rate': 0.8243319627833878, 'n_estimators': 154, 'reg_alpha': 357.1399618067286, 'reg_lambda': 0.1874352709490666}. Best is trial 4 with value: -0.9718200808825319.
[I 2023-04-25 20:43:27,763] Trial 41 finished with value: -1.2481987927662002 and parameters: {'max_depth': 45, 'num_leaves': 14, 'learning_rate': 0.007081079425795596, 'n_estimators': 82, 'reg_alpha': 38326.75021571517, 'reg_lambda': 25.577990221645408}. Best is trial 4 with value: -0.9718200808825319.
[I 2023-04-25 20:43:30,992] Trial 40 finished with value: -1.0228386451367353 and para

[I 2023-04-25 20:44:41,118] Trial 12 finished with value: -2.4821282122958515 and parameters: {'max_depth': 39, 'num_leaves': 20, 'learning_rate': 0.00010919450739471236, 'n_estimators': 189, 'reg_alpha': 4.23444701073929e-06, 'reg_lambda': 317569.67935870634}. Best is trial 7 with value: -2.4553536923657777.
[I 2023-04-25 20:44:44,818] Trial 13 finished with value: -2.8756547896241553 and parameters: {'max_depth': 0, 'num_leaves': 13, 'learning_rate': 0.1289507146279756, 'n_estimators': 166, 'reg_alpha': 1.264736604493388e-07, 'reg_lambda': 134.7073936262077}. Best is trial 7 with value: -2.4553536923657777.
[I 2023-04-25 20:44:49,854] Trial 9 finished with value: -2.4423871705634723 and parameters: {'max_depth': 35, 'num_leaves': 32, 'learning_rate': 0.0010130643531487764, 'n_estimators': 427, 'reg_alpha': 977.3523437723962, 'reg_lambda': 0.0003210304450351808}. Best is trial 9 with value: -2.4423871705634723.
[I 2023-04-25 20:44:56,880] Trial 14 finished with value: -2.4732117226975

[I 2023-04-25 20:47:46,981] Trial 36 finished with value: -2.4419645601120346 and parameters: {'max_depth': 33, 'num_leaves': 29, 'learning_rate': 0.0007514681418119728, 'n_estimators': 382, 'reg_alpha': 2.680803068005249, 'reg_lambda': 0.0021082704297634525}. Best is trial 26 with value: -2.440995292882166.
[I 2023-04-25 20:47:51,291] Trial 41 finished with value: -2.482840734393983 and parameters: {'max_depth': 47, 'num_leaves': 28, 'learning_rate': 8.677704764663178e-05, 'n_estimators': 456, 'reg_alpha': 42458.11280900164, 'reg_lambda': 2.093011893360949e-05}. Best is trial 26 with value: -2.440995292882166.
[I 2023-04-25 20:47:52,767] Trial 40 finished with value: -2.481087361599238 and parameters: {'max_depth': 47, 'num_leaves': 32, 'learning_rate': 5.7895850455702966e-05, 'n_estimators': 314, 'reg_alpha': 20408.062455187403, 'reg_lambda': 0.22150488025318682}. Best is trial 26 with value: -2.440995292882166.
[I 2023-04-25 20:47:54,007] Trial 39 finished with value: -2.47901233465

#### Dump the models

In [26]:
for key in models.keys():
    if models[key]:
        with open(f"./models/v2/{key}.pkl", "wb") as f:
            pickle.dump(models[key].best_estimator_, f)

#### Load all the models

In [27]:
model_files = sorted(glob('models/v2/*.pkl'))
estimators = []
for model_file in model_files:
    estimators.append((re.search(r"/(\d+_\d+)\.pkl$", model_file).group(1) ,pickle.load(open(model_file,'rb'))))

In [28]:
estimators_dict = dict(estimators)

In [29]:
def predict_using_individual_estimators(X_test, y_test, estimators_dict):
    results_df = pd.DataFrame(columns=['y_true', 'y_pred'])
    for site_meter in estimators_dict.keys():

        site_id, meter = list(map(lambda x:int(x) ,site_meter.split("_")))
        # Filter input data for the specific site_id and meter
        site_meter_filter = (X_test['site_id'] == site_id) & (X_test['meter'] == meter)

        X_test_subset = X_test[site_meter_filter].drop(['meter', 'site_id', 'building_id'], axis=1)

        # Get the true values for the subset of input data
        y_true = y_test[site_meter_filter]

        if(X_test_subset.shape[0] == 0):
            continue

        # Make predictions for the subset of input data
        y_pred = estimators_dict[site_meter].predict(X_test_subset)

        # Add true values and predictions to results_df
        results_df = pd.concat([results_df, pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})], ignore_index=True)

        # Compute RMSE for the predictions and print the results
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        print(f"Site ID: {site_id}, Meter: {meter}, RMSE: {rmse}")
    overall_rmse = mean_squared_error(results_df["y_true"], results_df["y_pred"], squared=False)
    print(f"Overall RMSE: {overall_rmse}")
    return results_df

In [30]:
results_df = predict_using_individual_estimators(X_test, y_test, estimators_dict)

Site ID: 0, Meter: 0, RMSE: 0.8958462019889623
Site ID: 0, Meter: 1, RMSE: 2.1661646078983527
Site ID: 10, Meter: 0, RMSE: 0.6435512253155503
Site ID: 10, Meter: 1, RMSE: 1.6477769444738206
Site ID: 10, Meter: 3, RMSE: 3.711565236249415
Site ID: 11, Meter: 0, RMSE: 0.8100198832108814
Site ID: 11, Meter: 1, RMSE: 1.0053629865051774
Site ID: 12, Meter: 0, RMSE: 0.8324606322487913
Site ID: 13, Meter: 0, RMSE: 1.0598314861107465
Site ID: 13, Meter: 1, RMSE: 1.8940835594066505
Site ID: 13, Meter: 2, RMSE: 3.0453410591345853
Site ID: 14, Meter: 0, RMSE: 1.2125531676504986
Site ID: 14, Meter: 1, RMSE: 2.141097753867811
Site ID: 14, Meter: 2, RMSE: 2.5810472416465533
Site ID: 14, Meter: 3, RMSE: 2.4446364198503847
Site ID: 15, Meter: 0, RMSE: 1.0624144124662596
Site ID: 15, Meter: 1, RMSE: 1.6748270926643025
Site ID: 15, Meter: 2, RMSE: 1.87421251439107
Site ID: 1, Meter: 0, RMSE: 1.4796705873505094
Site ID: 1, Meter: 3, RMSE: 2.431672659759725
Site ID: 2, Meter: 0, RMSE: 0.969575755492034
Sit

In [31]:
results_df = predict_using_individual_estimators(X_train, y_train, estimators_dict)

Site ID: 0, Meter: 0, RMSE: 0.4370975006650308
Site ID: 0, Meter: 1, RMSE: 2.1872053854320357
Site ID: 10, Meter: 0, RMSE: 0.6082265475033243
Site ID: 10, Meter: 1, RMSE: 1.8204013366918306
Site ID: 10, Meter: 3, RMSE: 2.035142421851524
Site ID: 11, Meter: 0, RMSE: 0.048857680003833164
Site ID: 11, Meter: 1, RMSE: 0.940267095748208
Site ID: 11, Meter: 3, RMSE: 1.0118834799747936
Site ID: 12, Meter: 0, RMSE: 0.6368874353024025
Site ID: 13, Meter: 0, RMSE: 0.6528128522219188
Site ID: 13, Meter: 1, RMSE: 1.5982944312383787
Site ID: 13, Meter: 2, RMSE: 1.8684257375013353
Site ID: 14, Meter: 0, RMSE: 1.088136540136423
Site ID: 14, Meter: 1, RMSE: 1.782384007456194
Site ID: 14, Meter: 2, RMSE: 2.4408351029646567
Site ID: 14, Meter: 3, RMSE: 2.227791638106526
Site ID: 15, Meter: 0, RMSE: 0.7422330514745354
Site ID: 15, Meter: 1, RMSE: 1.1990100136117974
Site ID: 15, Meter: 2, RMSE: 1.3754276357079414
Site ID: 15, Meter: 3, RMSE: 1.1578740694339213
Site ID: 1, Meter: 0, RMSE: 1.052804311683834

In [55]:
best_params_df = pd.DataFrame()
for key in models.keys():
    if models[key]:
        try:
            params = models[key].best_params_
            params["best_score"] = models[key].best_score_
        except:
            params = {}
        params["site_id"] = key.split("_")[0]
        params["meter"] = key.split("_")[1]
        best_params_df = pd.concat([best_params_df, pd.DataFrame(params, index=[key])])

In [56]:
best_params_df

,max_depth,num_leaves,learning_rate,n_estimators,reg_alpha,reg_lambda,best_score,site_id,meter
1_0,42.0,5.0,0.001265,282.0,1.008666e-04,9.375840e-02,-1.131032,1,0
1_3,12.0,21.0,0.000779,444.0,2.557156e-01,2.064281e-01,-2.123030,1,3
2_0,2.0,47.0,0.190963,330.0,1.437325e+00,4.692658e+05,-0.943938,2,0
2_3,1.0,34.0,0.013899,161.0,5.014057e-04,1.570336e-03,-1.613363,2,3
2_1,25.0,7.0,0.016610,256.0,1.100183e-02,2.201556e-06,-1.653434,2,1
3_0,-1.0,15.0,0.377659,91.0,8.965814e-07,8.689344e+05,-0.913410,3,0
4_0,26.0,28.0,0.188002,392.0,7.462068e+02,1.416566e-01,-0.869348,4,0
5_0,31.0,16.0,0.068284,218.0,3.058060e+03,5.408897e-01,-0.932333,5,0
6_0,12.0,34.0,0.084916,499.0,1.057952e-07,9.799330e+05,-0.900117,6,0
6_1,2.0,8.0,0.001208,400.0,6.107786e+01,1.910842e-06,-2.553161,6,1


In [61]:
models["11_0"].trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_max_depth,params_n_estimators,params_num_leaves,params_reg_alpha,...,user_attrs_mean_score_time,user_attrs_mean_test_score,user_attrs_split0_test_score,user_attrs_split1_test_score,user_attrs_split2_test_score,user_attrs_split3_test_score,user_attrs_std_fit_time,user_attrs_std_score_time,user_attrs_std_test_score,state
0,0,None,2023-04-25 19:15:52.843865,2023-04-25 19:15:57.533979,0 days 00:00:04.690114,2.853813e-06,43,385,18,9.046719e+02,...,0.000227,NaN,NaN,NaN,NaN,NaN,0.681406,0.000168,NaN,FAIL
1,1,None,2023-04-25 19:15:52.850513,2023-04-25 19:16:04.086423,0 days 00:00:11.235910,6.392680e-03,39,298,24,1.626622e-01,...,0.000510,NaN,NaN,NaN,NaN,NaN,1.613155,0.000602,NaN,FAIL
2,2,None,2023-04-25 19:15:52.852508,2023-04-25 19:15:54.727451,0 days 00:00:01.874943,7.453219e-03,34,415,46,2.997876e+05,...,0.000710,NaN,NaN,NaN,NaN,NaN,0.278675,0.000984,NaN,FAIL
3,3,None,2023-04-25 19:15:52.854734,2023-04-25 19:16:20.864823,0 days 00:00:28.010089,3.160354e-03,14,443,35,3.560444e+00,...,0.000177,NaN,NaN,NaN,NaN,NaN,4.073371,0.000104,NaN,FAIL
4,4,None,2023-04-25 19:15:54.730126,2023-04-25 19:15:55.835744,0 days 00:00:01.105618,1.172477e-02,3,70,25,2.412308e-05,...,0.000223,NaN,NaN,NaN,NaN,NaN,0.168333,0.000162,NaN,FAIL
5,5,None,2023-04-25 19:15:55.838492,2023-04-25 19:15:59.059942,0 days 00:00:03.221450,8.678914e-04,42,268,50,1.215929e+03,...,0.000179,NaN,NaN,NaN,NaN,NaN,0.485249,0.000106,NaN,FAIL
6,6,None,2023-04-25 19:15:57.536587,2023-04-25 19:16:13.816861,0 days 00:00:16.280274,7.936785e-04,31,239,37,2.898895e-06,...,0.000162,NaN,NaN,NaN,NaN,NaN,2.354601,0.000094,NaN,FAIL
7,7,None,2023-04-25 19:15:59.062499,2023-04-25 19:16:07.901796,0 days 00:00:08.839297,1.353845e-02,30,298,16,1.704116e-06,...,0.000538,NaN,NaN,NaN,NaN,NaN,1.276135,0.000653,NaN,FAIL
8,8,None,2023-04-25 19:16:04.088592,2023-04-25 19:16:04.250656,0 days 00:00:00.162064,2.408493e-04,31,40,46,8.211615e+04,...,0.000193,NaN,NaN,NaN,NaN,NaN,0.022451,0.000114,NaN,FAIL
9,9,None,2023-04-25 19:16:04.252953,2023-04-25 19:16:17.565311,0 days 00:00:13.312358,1.024572e-04,39,459,19,1.019510e+01,...,0.000165,NaN,NaN,NaN,NaN,NaN,1.922425,0.000095,NaN,FAIL


In [67]:
trials_df = pd.DataFrame()
for key in models.keys():
    if models[key]:
        trial_df = models[key].trials_dataframe()
        trial_df["key"] = key
        trials_df = pd.concat([trials_df, trial_df])

In [70]:
trials_df.to_csv("OptunaStudyTrialsAll.csv", index=False)